# B1 — Micro‑TSP para Inspeção de Waypoints (Médio)

Ordene ≤ 10 waypoints para minimizar o comprimento do tour. Incentiva uso de QAOA sobre um QUBO de TSP; baseline clássico fornecido.

### Contexto
Robôs móveis de inspeção frequentemente revisitam pequenos agrupamentos de waypoints (≤10 locais) para checar equipamentos. Com bateria limitada, planejam um **micro-TSP** usando distâncias localizadas. [`common.data_utils.micro_tsp_instance`](common/data_utils.py) retorna:
- `coords`: posições inteiras em grade para cada waypoint (indexação a partir de 0)
- `obstacles`: máscara booleana de grade (`True` ⇒ célula bloqueada) que pode inspirar penalidades ou reponderação (não usada pelo grader)
- `D`: matriz de distâncias Euclidianas usada por baselines e pelo grader

### Por que quântico?
TSPs minúsculos cabem em um QUBO com ≤100 variáveis binárias, adequado para testar QAOA ou heurísticas de annealing quântico. Amostragem quântica pode diversificar tours candidatos além do guloso + 2-opt, potencialmente gerando caminhos mais curtos sob orçamentos de tempo apertados.

## Especificação de Entrada e Saída
**Entrada:** `D (n×n)` matriz de distâncias (n≤10)  
**Saída:** `tour (list[int])` permutação dos nós 0..n-1  
Orientações de recursos/tempo: qubits ≤ 10, passos do otimizador ≤ 150; manter wall-time do solver em poucos segundos.

## Setup

In [ ]:
import numpy as np
from common import data_utils as du
from common import baselines as bl
from common import quantum_utils as qu
np.random.seed(1337)


_Orientação de baseline:_ [`common.baselines.baseline_tsp`](common/baselines.py) gera um tour inicial guloso e aplica 2-opt. Nas instâncias geradas fica ≈5–8% do ótimo; testes públicos exigem tour ≤ 1.15× baseline.

## Baseline (referência)

In [ ]:
coords, obstacles, D = du.micro_tsp_instance(n_nodes=8, grid=10, seed=0)
tour = bl.baseline_tsp(D)
def tour_len(t):
    return sum(D[t[i], t[(i+1)%len(t)]] for i in range(len(t)))
print("Comprimento do tour baseline:", tour_len(tour))


## Sua tarefa: implementar `solve(...)`

In [ ]:
def solve(D):
    # Esboço: codificar D em um QUBO de TSP (variáveis binárias x_{i,p}), rodar QAOA / annealer quântico,
    # arredondar para uma permutação e aplicar um polimento 2-opt rápido. Garantir visita única a cada nó.
    return bl.baseline_tsp(D)

## Testes públicos (rápidos)

In [ ]:
coords, obstacles, D = du.micro_tsp_instance(n_nodes=9, grid=10, seed=1)
t = solve(D)
def L(t):
    return sum(D[t[i], t[(i+1)%len(t)]] for i in range(len(t)))
L_base = L(bl.baseline_tsp(D))
print("Seu comprimento:", L(t), "Comprimento baseline:", L_base)
assert set(t) == set(range(D.shape[0]))
assert L(t) <= 1.15 * L_base + 1e-6, "Tour muito longo em relação ao baseline"
print("OK")


> Testes adicionais serão executados pelos organizadores com seeds/tamanhos diferentes.